In [1]:
try:
    import tensorflow as tf
    from keras.models import Sequential, save_model, load_model
    from keras.layers import Dense, LSTM, Input, Dropout
    from keras.callbacks import EarlyStopping
    from padasip.filters import FilterRLS
    from sklearn.multioutput import MultiOutputRegressor
    from lightgbm import LGBMRegressor
    import pandas as pd
    import numpy as np
    from datetime import datetime, timedelta, timezone
    import json
    import pickle
    import os
    import sys
    import argparse
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    from DB_Utils import select_model_for_forecast, load_artifact_from_storage, store_forecasts, fetch_data
    from supabase import create_client, Client
    import traceback
    import plotly.express as px
    from dotenv import load_dotenv, find_dotenv

    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"  # Or '2' or '3'

    KERAS_AVAILABLE = True
    PADASIP_AVAILABLE = True

    np.set_printoptions(suppress=True)

    load_dotenv()
    print("Env file found at location: ", find_dotenv())

except ImportError:
    print("TensorFlow/Keras not found. Keras models cannot be trained/saved natively.")
    KERAS_AVAILABLE = False
    # Define dummy classes if needed for type checking, though not strictly necessary here
    Sequential, save_model, load_model = object, lambda x, y: None, lambda x: None
    Dense, LSTM, Input, Dropout, EarlyStopping = object, object, object, object, object

    print("padasip not found. RLS filters cannot be loaded/used.")
    PADASIP_AVAILABLE = False
    FilterRLS = object


# --- Configuration & Constants --- (Same as before)
SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_SECRET_KEY")
if not SUPABASE_URL or not SUPABASE_KEY or "YOUR_SUPABASE_URL" in SUPABASE_URL:
    print("Error: SUPABASE_URL and SUPABASE_KEY environment variables must be set.")
    # sys.exit(1)

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
DATA_SCHEMA = "data"
ML_SCHEMA = "ml"
STORAGE_BUCKET = "models"
FEEDER_ID_TO_TRAIN = 1
SCENARIO = "Day"  # Example
MODEL_VERSION = "v1.1_Is_Weekend"  # Updated version
TRAIN_START_DATE = "2024-01-01 00:00:00+00"
TRAIN_END_DATE = "2024-05-31 23:59:59+00"
VALIDATION_START_DATE = "2024-06-01 00:00:00+00"
VALIDATION_END_DATE = "2024-06-30 23:59:59+00"
DAY_HOURS = list(range(6, 20 + 1))
NIGHT_HOURS = list(range(0, 6)) + list(range(21, 24))
script_dir = os.path.dirname(os.path.abspath(__file__)) if "__file__" in locals() else os.getcwd()
TEMP_DIR = os.path.join(script_dir, "tmp")

Env file found at location:  h:\My Drive\Barbados_Forecasting_Tool_Final\.env
Env file found at location:  h:\My Drive\Barbados_Forecasting_Tool_Final\.env


In [23]:
supabase.postgrest.schema(ML_SCHEMA)
lightgbm_results = supabase.table("forecasts").select("target_timestamp, forecast_value, actual_value").eq("feeder_id", 1).eq("model_id", 1).gte("target_timestamp", '2024-07-03').execute()
lstm_rls_results = supabase.table("forecasts").select("target_timestamp, forecast_value").eq("feeder_id", 1).eq("model_id", 19).gte("target_timestamp", '2024-07-03').execute()

lightgbm_results_df = pd.DataFrame(lightgbm_results.data).set_index("target_timestamp")
lstm_rls_results_df = pd.DataFrame(lstm_rls_results.data).set_index("target_timestamp")

lightgbm_results_df.rename(columns={"forecast_value": "lightgbm_forecast"}, inplace=True)
lstm_rls_results_df.rename(columns={"forecast_value": "lstm_rls_forecast"}, inplace=True)

px.line(pd.concat([lightgbm_results_df, lstm_rls_results_df], axis=1))
# select * from ml.forecasts where Feeder_ID = 1 and target_timestamp >= '2024-07-03' and model_id = 19